# Performance Masking

## Description
When agents act in such a way, through their dealing with or knowledge of IT systems, that their true performance cannot (easily) be established.

In [1]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable('vegafusion')

DataTransformerRegistry.enable('vegafusion')

In [2]:
log_name = 'propr_test'

fn_logs = {
    'propr': '../data/preproc/proprietary.csv',
    'propr_ref': '../data/preproc/propr_train-70.csv',
    'propr_test': '../data/preproc/propr_test-30.csv'
}

In [3]:
log = pd.read_csv(fn_logs[log_name])
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp
0,Precheck,ID-2,6/05/2011 16:09,6/05/2011 16:27,090-10-02,Friday,"0,87 € per minute",0.0,1.0,0.0,2011-05-06 16:09:00,2011-05-06 16:27:00
1,Application_Processing_Branches,ID-2,9/05/2011 11:14,9/05/2011 11:16,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 11:14:00,2011-05-09 11:16:00
2,Precheck,ID-2,9/05/2011 15:44,9/05/2011 15:58,090-10-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-05-09 15:44:00,2011-05-09 15:58:00
3,Processing_Incomplete_Orders,ID-2,9/05/2011 16:10,9/05/2011 16:10,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 16:10:00,2011-05-09 16:10:00
4,Application_Processing_Branches,ID-2,10/05/2011 8:52,10/05/2011 9:32,010-23-13,Tuesday,"1,02 € per minute",0.0,1.0,0.0,2011-05-10 08:52:00,2011-05-10 09:32:00
...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Precheck,ID-1720,20/05/2011 17:24,20/05/2011 17:26,000-3-01,Friday,"0,87 € per minute",0.0,2.0,0.0,2011-05-20 17:24:00,2011-05-20 17:26:00
5286,Check_of_Processing_Applications,ID-1720,23/05/2011 15:53,23/05/2011 16:06,000-2-01,Monday,"0,87 € per minute",0.0,2.0,0.0,2011-05-23 15:53:00,2011-05-23 16:06:00
5287,Processing_of_Applications,ID-1720,26/05/2011 9:25,26/05/2011 9:25,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 09:25:00,2011-05-26 09:25:00
5288,Archieving,ID-1720,26/05/2011 10:38,26/05/2011 10:47,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 10:38:00,2011-05-26 10:47:00


In [4]:
# encode activity labels as short names for presentation purpose

print('There exist {} unique activity labels'.format(
    log['activity'].nunique()
))

# use initial letters for encoding
act_labels_short = dict()
for label in log['activity'].unique():
    label_short = ''.join([w[0] for w in label.split('_')])
    act_labels_short[label] = label_short
print('There are {} short labels after encoding'.format(
    len(act_labels_short.values())
))
for k in sorted(act_labels_short.keys()):
    print('{:>50} -> {:>30}'.format(k, act_labels_short[k]))

log['activity_short'] = log['activity'].apply(
    lambda x: act_labels_short[x]
)

# add activity duration
log['start timestamp'] = pd.to_datetime(log['start timestamp'], format='mixed', dayfirst=True)
log['complete timestamp'] = pd.to_datetime(log['complete timestamp'], 
format='mixed', dayfirst=True)
# duration measured in minutes
log['activity duration'] = (
    log['complete timestamp'] - log['start timestamp']
).dt.total_seconds() / 60

log

There exist 25 unique activity labels
There are 25 short labels after encoding
                              Administrative_Tasks ->                             AT
                   Application_Processing_Branches ->                            APB
                                          Approval ->                              A
                                   Approval_Branch ->                             AB
                          Approval_Executive_Board ->                            AEB
                                        Archieving ->                              A
                                 Check_of_Approval ->                            CoA
                                Check_of_Documents ->                            CoD
                  Check_of_Processing_Applications ->                           CoPA
                                               End ->                              E
                                   Further_inquiry ->                  

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp,activity_short,activity duration
0,Precheck,ID-2,2011-05-06 16:09:00,2011-05-06 16:27:00,090-10-02,Friday,"0,87 € per minute",0.0,1.0,0.0,2011-05-06 16:09:00,2011-05-06 16:27:00,P,18.0
1,Application_Processing_Branches,ID-2,2011-05-09 11:14:00,2011-05-09 11:16:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 11:14:00,2011-05-09 11:16:00,APB,2.0
2,Precheck,ID-2,2011-05-09 15:44:00,2011-05-09 15:58:00,090-10-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-05-09 15:44:00,2011-05-09 15:58:00,P,14.0
3,Processing_Incomplete_Orders,ID-2,2011-05-09 16:10:00,2011-05-09 16:10:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 16:10:00,2011-05-09 16:10:00,PIO,0.0
4,Application_Processing_Branches,ID-2,2011-05-10 08:52:00,2011-05-10 09:32:00,010-23-13,Tuesday,"1,02 € per minute",0.0,1.0,0.0,2011-05-10 08:52:00,2011-05-10 09:32:00,APB,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Precheck,ID-1720,2011-05-20 17:24:00,2011-05-20 17:26:00,000-3-01,Friday,"0,87 € per minute",0.0,2.0,0.0,2011-05-20 17:24:00,2011-05-20 17:26:00,P,2.0
5286,Check_of_Processing_Applications,ID-1720,2011-05-23 15:53:00,2011-05-23 16:06:00,000-2-01,Monday,"0,87 € per minute",0.0,2.0,0.0,2011-05-23 15:53:00,2011-05-23 16:06:00,CoPA,13.0
5287,Processing_of_Applications,ID-1720,2011-05-26 09:25:00,2011-05-26 09:25:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 09:25:00,2011-05-26 09:25:00,PoA,0.0
5288,Archieving,ID-1720,2011-05-26 10:38:00,2011-05-26 10:47:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 10:38:00,2011-05-26 10:47:00,A,9.0


In [5]:
# proprietary
log['department'] = log['resource'].apply(
    lambda x: x.split('-')[0]
)
log['department_role'] = log['resource'].apply(
    lambda x: x.split('-')[0] + '-' + x.split('-')[1]
)
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp,activity_short,activity duration,department,department_role
0,Precheck,ID-2,2011-05-06 16:09:00,2011-05-06 16:27:00,090-10-02,Friday,"0,87 € per minute",0.0,1.0,0.0,2011-05-06 16:09:00,2011-05-06 16:27:00,P,18.0,090,090-10
1,Application_Processing_Branches,ID-2,2011-05-09 11:14:00,2011-05-09 11:16:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 11:14:00,2011-05-09 11:16:00,APB,2.0,010,010-23
2,Precheck,ID-2,2011-05-09 15:44:00,2011-05-09 15:58:00,090-10-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-05-09 15:44:00,2011-05-09 15:58:00,P,14.0,090,090-10
3,Processing_Incomplete_Orders,ID-2,2011-05-09 16:10:00,2011-05-09 16:10:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-05-09 16:10:00,2011-05-09 16:10:00,PIO,0.0,010,010-23
4,Application_Processing_Branches,ID-2,2011-05-10 08:52:00,2011-05-10 09:32:00,010-23-13,Tuesday,"1,02 € per minute",0.0,1.0,0.0,2011-05-10 08:52:00,2011-05-10 09:32:00,APB,40.0,010,010-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Precheck,ID-1720,2011-05-20 17:24:00,2011-05-20 17:26:00,000-3-01,Friday,"0,87 € per minute",0.0,2.0,0.0,2011-05-20 17:24:00,2011-05-20 17:26:00,P,2.0,000,000-3
5286,Check_of_Processing_Applications,ID-1720,2011-05-23 15:53:00,2011-05-23 16:06:00,000-2-01,Monday,"0,87 € per minute",0.0,2.0,0.0,2011-05-23 15:53:00,2011-05-23 16:06:00,CoPA,13.0,000,000-2
5287,Processing_of_Applications,ID-1720,2011-05-26 09:25:00,2011-05-26 09:25:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 09:25:00,2011-05-26 09:25:00,PoA,0.0,010,010-23
5288,Archieving,ID-1720,2011-05-26 10:38:00,2011-05-26 10:47:00,010-23-07,Thursday,"1,02 € per minute",0.0,2.0,0.0,2011-05-26 10:38:00,2011-05-26 10:47:00,A,9.0,010,010-23


## Performance-PM-2

In [6]:
sel_resources = log['resource'].unique()
print('{} resources are selected'.format(len(sel_resources)))

166 resources are selected


### For each resource, extract a set of activity sequences performed by this resource

In [7]:
# sort events by complete timestamps
log = log.sort_values(by='complete timestamp')

seq_log = []

for res, events in log.groupby('resource'):
    if res in sel_resources:
        for case_id, trace in events.groupby('case_id'):
            # ignore singletons
            if len(trace) <= 1:
                pass
            else:
                for i in range(len(trace) - 1):
                    for j in range(i + 1, len(trace)):
                        prefix = trace.iloc[i:j+1]
                        seq = ','.join(prefix['activity_short'])
                        length = len(prefix['activity_short'])
                        total_duration = prefix['activity duration'] = (
                            prefix.iloc[-1]['complete timestamp'] -
                            prefix.iloc[0]['start timestamp']
                        ).total_seconds() / 60
                        seq_log.append({
                            'seq': seq,
                            'resource': res,
                            'case_id': case_id,
                            'total_duration': total_duration
                        })

C:\Users\yangj29\AppData\Local\Temp\ipykernel_18096\4110256220.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_duration = prefix['activity duration'] = (
C:\Users\yangj29\AppData\Local\Temp\ipykernel_18096\4110256220.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_duration = prefix['activity duration'] = (
C:\Users\yangj29\AppData\Local\Temp\ipykernel_18096\4110256220.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [8]:
seq_log = pd.DataFrame(seq_log)
seq_log['length'] = seq_log['seq'].apply(lambda x: len(x.split(',')))
seq_log

,seq,resource,case_id,total_duration,length
0,"PIO,PIO",000-2-01,ID-1003,1160.0,2
1,"PIO,PIO,APB",000-2-01,ID-1003,11007.0,3
2,"PIO,APB",000-2-01,ID-1003,9847.0,2
3,"PIO,PIO",000-2-01,ID-1142,1390.0,2
4,"PIO,PIO,APB",000-2-01,ID-1142,2997.0,3
...,...,...,...,...,...
8628,"T,E,APB",111-9-1,ID-597,0.0,3
8629,"E,APB",111-9-1,ID-597,0.0,2
8630,"CoPA,Fi",33-01,ID-704,43.0,2
8631,"CoPA,Fi,PR",33-01,ID-704,83.0,3


In [9]:
print(seq_log['seq'].nunique())
print(seq_log['case_id'].nunique())
print(seq_log['resource'].nunique())

print(seq_log['length'].min())
print(seq_log['length'].max())
print(seq_log['total_duration'].min())
print(seq_log['total_duration'].max())


2318
499
145
2
18
0.0
105119.0


### Identify sequences that are long and performed in a short time

In [10]:
seq_low_threshold = []
for seq, stats in seq_log.groupby('seq'):
    lb = stats['total_duration'].quantile(0.25) - 1.5 * (
        stats['total_duration'].quantile(0.75) -
        stats['total_duration'].quantile(0.25)
    )
    seq_low_threshold.append({'seq': seq, 'lb': lb})

seq_low_threshold = pd.DataFrame(seq_low_threshold)

seq_log = seq_log.merge(seq_low_threshold, how='left', on='seq')
seq_log['is_short_duration'] = seq_log['total_duration'] < seq_log['lb']

seq_log

,seq,resource,case_id,total_duration,length,lb,is_short_duration
0,"PIO,PIO",000-2-01,ID-1003,1160.0,2,-2169.000,False
1,"PIO,PIO,APB",000-2-01,ID-1003,11007.0,3,-6578.000,False
2,"PIO,APB",000-2-01,ID-1003,9847.0,2,-2785.625,False
3,"PIO,PIO",000-2-01,ID-1142,1390.0,2,-2169.000,False
4,"PIO,PIO,APB",000-2-01,ID-1142,2997.0,3,-6578.000,False
...,...,...,...,...,...,...,...
8628,"T,E,APB",111-9-1,ID-597,0.0,3,0.000,False
8629,"E,APB",111-9-1,ID-597,0.0,2,0.000,False
8630,"CoPA,Fi",33-01,ID-704,43.0,2,-609.000,False
8631,"CoPA,Fi,PR",33-01,ID-704,83.0,3,83.000,False


In [11]:
short_seqs = set(
    seq_log.loc[
        (seq_log['is_short_duration'] == True) &
        (seq_log['total_duration'] < 60),
        'seq'
    ].unique()
)
print(len(short_seqs))

4


In [12]:
# alt.layer(
#     alt.Chart(seq_log[seq_log['seq'] == 'GSoC,PoA,A']).mark_boxplot(outliers={'size': 0}).encode(
#         x='total_duration:Q',
#         y='seq:N',
#         color=alt.value('grey'),
#         opacity=alt.value(0.5)
#     ),
#     alt.Chart(seq_log[seq_log['seq'] == 'GSoC,PoA,A']).mark_circle().encode(
#         x='total_duration:Q',
#         y='seq:N',
#         color='is_short_duration:N'
#     ),
# )

charts = []
for seq in short_seqs:
    data = seq_log[seq_log['seq'] == seq]
    charts.append(
        alt.layer(
            alt.Chart(data).mark_boxplot(outliers={'size': 0}).encode(
                x=alt.X('total_duration:Q').title('total duration (minutes)'),
                y=alt.Y('seq:N').title(['activity', 'sequence']),
                color=alt.value('grey'),
                opacity=alt.value(0.5)
            ),
            alt.Chart(data).mark_circle().encode(
                x=alt.X('total_duration:Q').title('total duration (minutes)'),
                y=alt.Y('seq:N').title(['activity', 'sequence']),
                color=alt.Color('is_short_duration:N').title([
                    'is significantly', 'short duration?'
                ]),
                tooltip=['resource', 'case_id', 'total_duration']
            )
        )
    )

alt.vconcat(*charts).resolve_scale(x='independent')

alt.VConcatChart(...)

In [13]:
charts = []
for seq in short_seqs:
    data = seq_log[seq_log['seq'] == seq].groupby('seq').agg(
        num_res=pd.NamedAgg('resource', 'nunique')
    )
    charts.append(
        alt.Chart(data).mark_bar().encode(
            x=alt.X('num_res:Q').title('number of resources'),
            y=alt.Y('seq:N').title(['activity', 'sequence'])
        )
    )

alt.vconcat(*charts).resolve_scale(x='shared')

alt.VConcatChart(...)